# Building Data Pipelines for ETL
I'll be using the "Google Play Store Apps" dataset, other files are attched. Those two datasets are made available as `.csv` files, and will be transformed before being loaded into a `sqlite` database.

# STEP 1 : Extracting Data
## Ways to extract data
- API's
- SFTP sites
- Relational databases
- NoSQL databases (columnar, document, key-value)
- Flat-files

In [16]:
# Import pandas
import pandas as pd
apps = pd.read_csv("apps_data.csv")
reviews = pd.read_csv("review_data.csv")

print(apps.head())
print(apps.columns)
print(apps.shape)
print(apps.dtypes)


                                                 App  ...   Android Ver
0     Photo Editor & Candy Camera & Grid & ScrapBook  ...  4.0.3 and up
1                                Coloring book moana  ...  4.0.3 and up
2  U Launcher Lite – FREE Live Cool Themes, Hide ...  ...  4.0.3 and up
3                              Sketch - Draw & Paint  ...    4.2 and up
4              Pixel Draw - Number Art Coloring Book  ...    4.4 and up

[5 rows x 13 columns]
Index(['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type',
       'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver',
       'Android Ver'],
      dtype='object')
(10841, 13)
App                object
Category           object
Rating            float64
Reviews            object
Size               object
Installs           object
Type               object
Price              object
Content Rating     object
Genres             object
Last Updated       object
Current Ver        object
Android Ver        object


In [20]:
import pandas as pd

# Writing a function to extract the data
def extract(file_path):
    data = pd.read_csv(file_path)
    
    # Print the details
    print(f"Here is a little bit of information about the data stored in {file_path}:")
    print(f"\nThere are {data.shape[0]} rows and {data.shape[1]} columns in this DataFrame.")
    print("\nThe columns in this DataFrame take the following types: ")
    
    # Print the type of each column
    print(data.dtypes)
    
    # Print a message before returning the DataFrame
    print(f"\nTo view the DataFrame extracted from {file_path}, display the value returned by this function!\n\n")
    
    return data
    
# Call the function
apps_data = extract("apps_data.csv")
reviews_data = extract("review_data.csv")

print('app_data')
print('reviews_data')

Here is a little bit of information about the data stored in apps_data.csv:

There are 10841 rows and 13 columns in this DataFrame.

The columns in this DataFrame take the following types: 
App                object
Category           object
Rating            float64
Reviews            object
Size               object
Installs           object
Type               object
Price              object
Content Rating     object
Genres             object
Last Updated       object
Current Ver        object
Android Ver        object
dtype: object

To view the DataFrame extracted from apps_data.csv, display the value returned by this function!


Here is a little bit of information about the data stored in review_data.csv:

There are 64295 rows and 5 columns in this DataFrame.

The columns in this DataFrame take the following types: 
App                        object
Translated_Review          object
Sentiment                  object
Sentiment_Polarity        float64
Sentiment_Subjectivity    float

# STEP 2 : Transforming Data

In [21]:
# Define a function to transform data
def transform(apps, reviews, category, min_rating, min_reviews):
    # Print statement for observability
    print(f"Transforming data to curate a dataset with all {category} apps and their "
          f"corresponding reviews with a rating of at least {min_rating} and "
          f"{min_reviews} reviews\n")
    
    # Drop any duplicates from both DataFrames
    reviews = reviews.drop_duplicates()
    apps = apps.drop_duplicates(["App"])
    
    # Find all of the apps and reviews in the food and drink category
    subset_apps = apps.loc[apps["Category"] == category, :]
    subset_reviews = reviews.loc[reviews["App"].isin(subset_apps["App"]), ["App", "Sentiment_Polarity"]]
    
    # Aggregate the subset_reviews
    aggregated_reviews = subset_reviews.groupby(by="App").mean()
    
    # Join it back to the subset_apps table
    joined_apps_reviews = subset_apps.join(aggregated_reviews, on="App", how="left")
    
    # Keep only the needed columns
    filtered_apps_reviews = joined_apps_reviews.loc[:, ["App", "Rating", "Reviews", "Installs", "Sentiment_Polarity"]]
    
    # Convert reviews, keep only values with an average rating of at least 4 stars, and at least 1000 reviews
    filtered_apps_reviews = filtered_apps_reviews.astype({"Reviews": "int32"})
    top_apps = filtered_apps_reviews.loc[(filtered_apps_reviews["Rating"] > min_rating) & (filtered_apps_reviews["Reviews"] > min_reviews), :]
    
    # Sort the top apps, replace NaN with 0, reset the index (drop, inplace)
    top_apps.sort_values(by=["Rating", "Reviews"], ascending=False, inplace=True)
    top_apps.reset_index(drop=True, inplace=True)
     
    # Persist this DataFrame as top_apps.csv file
    top_apps.to_csv("top_apps.csv")
    
    print(f"The transformed DataFrame, which includes {top_apps.shape[0]} rows "
          f"and {top_apps.shape[1]} columns has been persisted, and will now be "
          f"returned")
    
    # Return the transformed DataFrame
    return top_apps


# Call the function
top_apps_data = transform(
    apps=apps_data,
    reviews=reviews_data,
    category="FOOD_AND_DRINK",
    min_rating=4.0,
    min_reviews=1000
)

# Show
top_apps_data


Transforming data to curate a dataset with all FOOD_AND_DRINK apps and their corresponding reviews with a rating of at least 4.0 and 1000 reviews

The transformed DataFrame, which includes 54 rows and 5 columns has been persisted, and will now be returned


,App,Rating,Reviews,Installs,Sentiment_Polarity
0,SarashpazPapion (Cooking with Chef Bowls),4.8,1250,"50,000+",NaN
1,Domino's Pizza USA,4.7,1032935,"10,000,000+",0.226971
2,Tastely,4.7,611136,"10,000,000+",NaN
3,Delicious Recipes,4.7,129737,"1,000,000+",NaN
4,BeyondMenu Food Delivery,4.7,51517,"1,000,000+",0.408743
5,Recipes Pastries and homemade pies More than 5...,4.7,14065,"500,000+",NaN
6,Pastry & Cooking (Without Net),4.7,6118,"1,000,000+",NaN
7,Simple Recipes,4.7,3803,"500,000+",NaN
8,Easy Recipes,4.7,2707,"100,000+",0.284777
9,OpenTable: Restaurants Near Me,4.6,90242,"5,000,000+",NaN


# STEP 3 : Loading Data
## using `pandas` along with `sqlite` to do !

In [22]:
import sqlite3

# Now, create a function to do this
def load(dataframe, database_name, table_name):
    # Create a connection object
    con = sqlite3.connect(database_name)
    
    # Write the data to the specified table (table_name)
    dataframe.to_sql(name=table_name, con=con, if_exists="replace", index=False)
    print("Original DataFrame has been loaded to sqlite\n")
    
    # Read the data, and return the result
    loaded_dataframe = pd.read_sql(sql=f"SELECT * FROM {table_name}", con=con)
    print("The loaded DataFrame has been read from sqlite for validation\n")
    
    try:
        assert dataframe.shape == loaded_dataframe.shape
        print(f"Success! The data in the {table_name} table have successfully been "
              f"loaded and validated")

    except AssertionError:
        print("DataFrame shape is not consistent before and after loading. Take a closer look!")


# Call the function
load(
    dataframe=top_apps_data,
    database_name="market_research",
    table_name="top_apps"
)

Original DataFrame has been loaded to sqlite

The loaded DataFrame has been read from sqlite for validation

Success! The data in the top_apps table have successfully been loaded and validated


# FINAL STEP : Running the Pipeline

In [23]:
# Import modules
import pandas as pd
import sqlite3

# Extract the data
apps_data = extract("apps_data.csv")
reviews_data = extract("review_data.csv")

# Transform the data
top_apps_data = transform(
    apps=apps_data,
    reviews=reviews_data,
    category="FOOD_AND_DRINK",
    min_rating=3.0,
    min_reviews=1000
)

# Load the data
load(
    dataframe=top_apps_data,
    database_name="market_research",
    table_name="top_apps"
)


Here is a little bit of information about the data stored in apps_data.csv:

There are 10841 rows and 13 columns in this DataFrame.

The columns in this DataFrame take the following types: 
App                object
Category           object
Rating            float64
Reviews            object
Size               object
Installs           object
Type               object
Price              object
Content Rating     object
Genres             object
Last Updated       object
Current Ver        object
Android Ver        object
dtype: object

To view the DataFrame extracted from apps_data.csv, display the value returned by this function!


Here is a little bit of information about the data stored in review_data.csv:

There are 64295 rows and 5 columns in this DataFrame.

The columns in this DataFrame take the following types: 
App                        object
Translated_Review          object
Sentiment                  object
Sentiment_Polarity        float64
Sentiment_Subjectivity    float